# Interactive Cough Detection Model Tester

This notebook provides an interactive Gradio interface for testing XGBoost cough detection models trained on multimodal biosignals (audio + IMU).

## Features

- **Load pre-trained models**: IMU-only, Audio-only, and Multimodal classifiers
- **Test on dataset recordings**: Select from public_dataset with ground truth comparison
- **Upload custom files**: Test on your own WAV audio and CSV IMU data
- **Interactive visualizations**: Waveforms with color-coded detections
- **Threshold adjustment**: Fine-tune classification threshold in real-time
- **Event-based metrics**: TP/FP/FN counts with sensitivity, precision, F1 scores

## Prerequisites

**IMPORTANT**: You must first run `Model_Training_XGBoost.ipynb` to completion to generate the saved models in `models/`.

The training notebook should create:
- `models/xgb_imu.pkl`
- `models/xgb_audio.pkl`
- `models/xgb_multimodal.pkl`

## Section 1: Setup & Configuration

In [ ]:
# Check for required dependencies
import sys

try:
    import gradio as gr
    import xgboost
    import joblib
    print("✓ All required dependencies installed")
    print(f"  - gradio version: {gr.__version__}")
    print(f"  - xgboost version: {xgboost.__version__}")
except ImportError as e:
    print(f"✗ Missing dependency: {e}")
    print("\nInstall with: uv add gradio xgboost joblib")
    sys.exit(1)

In [ ]:
# Import dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from scipy.io import wavfile
from scipy import signal
import librosa
from sklearn.preprocessing import StandardScaler
import pickle
from pathlib import Path
import os
import warnings
warnings.filterwarnings('ignore')

# Add src directory to path
sys.path.append(os.path.abspath('../src'))
from helpers import *
from features import extract_audio_features, extract_imu_features

print("✓ All imports successful")

In [ ]:
# Set constants
FS_AUDIO_CONST = 16000  # Audio sampling frequency
FS_IMU_CONST = 100      # IMU sampling frequency
WINDOW_LEN = 0.4        # Window length in seconds
HOP_SIZE = 0.05         # Default hop size for sliding window (50ms)

# Locate dataset folder
kaggle_dataset_dir = '/kaggle/input/edge-ai-cough-count'
base_dir = kaggle_dataset_dir if os.path.exists(kaggle_dataset_dir) else ".."
data_folder = base_dir + '/public_dataset/'

if not os.path.exists(data_folder):
    raise FileNotFoundError(
        "Cannot find public_dataset/. Please download from: "
        "https://zenodo.org/record/7562332"
    )

# Locate models directory
MODEL_DIR = Path("./models")

print(f"Configuration:")
print(f"  Audio FS: {FS_AUDIO_CONST} Hz")
print(f"  IMU FS: {FS_IMU_CONST} Hz")
print(f"  Window length: {WINDOW_LEN}s")
print(f"  Dataset folder: {data_folder if data_folder else 'Not found'}")
print(f"  Models directory: {MODEL_DIR}")

## Section 2: Model Loading System

In [ ]:
def load_trained_models():
    """
    Load all three trained models from disk.
    
    Returns:
        dict: Dictionary with keys 'imu', 'audio', 'multimodal'
              Each value is {'model': XGBClassifier, 'scaler': StandardScaler, 'threshold': float}
    """
    models = {}
    
    for modality in ['imu', 'audio', 'multimodal']:
        model_path = MODEL_DIR / f'xgb_{modality}.pkl'
        
        if not model_path.exists():
            raise FileNotFoundError(
                f"\n{'='*70}\n"
                f"ERROR: Model file not found: {model_path}\n\n"
                f"Please run Model_Training_XGBoost.ipynb first to train and save models.\n"
                f"The training notebook should create the following files:\n"
                f"  - models/xgb_imu.pkl\n"
                f"  - models/xgb_audio.pkl\n"
                f"  - models/xgb_multimodal.pkl\n"
                f"{'='*70}"
            )
        
        with open(model_path, 'rb') as f:
            models[modality] = pickle.load(f)
        
        print(f"✓ Loaded {modality} model from {model_path}")
        print(f"  Threshold: {models[modality]['threshold']:.3f}")
    
    return models

# Load models
try:
    MODELS = load_trained_models()
    print(f"\n✓ All models loaded successfully")
except FileNotFoundError as e:
    print(e)
    MODELS = None

## Section 3: Feature Extraction Utilities

In [ ]:
def extract_features_for_window(audio_window, imu_window, modality='multimodal'):
    """
    Extract features from a single window of audio and IMU data.
    
    Args:
        audio_window: (N_audio,) audio samples
        imu_window: (N_imu, 6) IMU samples
        modality: 'imu', 'audio', or 'multimodal'
    
    Returns:
        np.array: Feature vector
    """
    features = []
    
    if modality in ['audio', 'multimodal']:
        audio_feat = extract_audio_features(audio_window, fs=FS_AUDIO_CONST)
        # Handle NaN/Inf
        audio_feat = np.nan_to_num(audio_feat, nan=0.0, posinf=0.0, neginf=0.0)
        features.append(audio_feat)
    
    if modality in ['imu', 'multimodal']:
        imu_feat = extract_imu_features(imu_window)
        # Handle NaN/Inf
        imu_feat = np.nan_to_num(imu_feat, nan=0.0, posinf=0.0, neginf=0.0)
        features.append(imu_feat)
    
    return np.concatenate(features)

print("✓ Feature extraction utilities ready")

## Section 4: Sliding Window Prediction Engine

In [ ]:
def sliding_window_predict(audio, imu, model_data, modality='multimodal', 
                          window_len=0.4, hop_size=0.05, threshold=None):
    """
    Apply model to continuous recording using sliding windows.
    
    Args:
        audio: (N_audio,) audio samples
        imu: (N_imu, 6) IMU samples
        model_data: Dict with 'model', 'scaler', 'threshold'
        modality: 'imu', 'audio', or 'multimodal'
        window_len: Window length in seconds
        hop_size: Hop size in seconds
        threshold: Classification threshold (None = use optimal from model)
    
    Returns:
        predictions: List of (start_time, end_time, probability) tuples
        all_probs: Array of probabilities for each window
        window_times: Array of window center times
    """
    model = model_data['model']
    scaler = model_data['scaler']
    if threshold is None:
        threshold = model_data['threshold']
    
    # Calculate window and hop in samples
    audio_win_samples = int(window_len * FS_AUDIO_CONST)
    audio_hop_samples = int(hop_size * FS_AUDIO_CONST)
    imu_win_samples = int(window_len * FS_IMU_CONST)
    imu_hop_samples = int(hop_size * FS_IMU_CONST)
    
    # Extract windows
    n_windows = (len(audio) - audio_win_samples) // audio_hop_samples + 1
    features_list = []
    window_times = []
    
    for i in range(n_windows):
        audio_start = i * audio_hop_samples
        audio_end = audio_start + audio_win_samples
        imu_start = i * imu_hop_samples
        imu_end = imu_start + imu_win_samples
        
        if audio_end > len(audio) or imu_end > len(imu):
            break
        
        audio_window = audio[audio_start:audio_end]
        imu_window = imu[imu_start:imu_end, :]
        
        features = extract_features_for_window(audio_window, imu_window, modality)
        features_list.append(features)
        
        # Window center time
        center_time = (audio_start + audio_win_samples / 2) / FS_AUDIO_CONST
        window_times.append(center_time)
    
    # Batch predict
    X = np.array(features_list)
    X_scaled = scaler.transform(X)
    probs = model.predict_proba(X_scaled)[:, 1]
    
    # Convert to event-based predictions
    predictions = []
    for i, (prob, center) in enumerate(zip(probs, window_times)):
        if prob >= threshold:
            start = center - window_len / 2
            end = center + window_len / 2
            predictions.append((start, end, prob))
    
    return predictions, probs, np.array(window_times)

print("✓ Sliding window prediction engine ready")

In [ ]:
def merge_detections(predictions, gap_threshold=0.3):
    """
    Merge consecutive detections that are close together.
    
    Args:
        predictions: List of (start, end, prob) tuples
        gap_threshold: Maximum gap between events to merge (seconds)
    
    Returns:
        merged: List of (start, end, max_prob) tuples
    """
    if not predictions:
        return []
    
    # Sort by start time
    sorted_preds = sorted(predictions, key=lambda x: x[0])
    
    merged = []
    current_start, current_end, current_prob = sorted_preds[0]
    
    for start, end, prob in sorted_preds[1:]:
        # If gap is small, merge
        if start - current_end <= gap_threshold:
            current_end = max(current_end, end)
            current_prob = max(current_prob, prob)
        else:
            # Save current event and start new one
            merged.append((current_start, current_end, current_prob))
            current_start, current_end, current_prob = start, end, prob
    
    # Add last event
    merged.append((current_start, current_end, current_prob))
    
    return merged

print("✓ Detection merging ready")

## Section 5: Visualization Functions

In [ ]:
def plot_predictions(audio, imu, predictions, ground_truth=None, figsize=(14, 8)):
    """
    Plot waveform with highlighted detections.
    
    Args:
        audio: (N,) audio samples
        imu: (M, 6) IMU samples
        predictions: List of (start, end, prob) tuples
        ground_truth: Optional list of (start, end) tuples for ground truth
        figsize: Figure size
    
    Returns:
        matplotlib.figure.Figure
    """
    fig, axes = plt.subplots(2, 1, figsize=figsize)
    
    # Audio time axis
    audio_time = np.arange(len(audio)) / FS_AUDIO_CONST
    
    # IMU time axis
    imu_time = np.arange(len(imu)) / FS_IMU_CONST
    
    # Plot audio
    axes[0].plot(audio_time, audio, linewidth=0.5, color='black', alpha=0.7)
    axes[0].set_ylabel('Amplitude', fontsize=11)
    axes[0].set_title('Audio Waveform (Outer Microphone)', fontsize=12, fontweight='bold')
    axes[0].grid(alpha=0.3)
    
    # Plot IMU (Z-axis, negated)
    axes[1].plot(imu_time, -imu[:, 2], linewidth=1, color='blue', alpha=0.7)
    axes[1].set_xlabel('Time (s)', fontsize=11)
    axes[1].set_ylabel('Acceleration', fontsize=11)
    axes[1].set_title('IMU Accelerometer Z (negated)', fontsize=12, fontweight='bold')
    axes[1].grid(alpha=0.3)
    
    # Add predictions as red spans
    for start, end, prob in predictions:
        for ax in axes:
            ax.axvspan(start, end, alpha=0.3, color='red', label='Prediction' if start == predictions[0][0] else '')
            # Add confidence label on audio plot
            if ax == axes[0]:
                mid = (start + end) / 2
                ax.text(mid, ax.get_ylim()[1] * 0.9, f'{prob:.2f}', 
                       ha='center', va='top', fontsize=9, color='red', fontweight='bold')
    
    # Add ground truth as green spans
    if ground_truth:
        for start, end in ground_truth:
            for ax in axes:
                ax.axvspan(start, end, alpha=0.2, color='green', 
                          label='Ground Truth' if start == ground_truth[0][0] else '')
    
    # Add legends
    for ax in axes:
        handles, labels = ax.get_legend_handles_labels()
        if handles:
            # Remove duplicates
            by_label = dict(zip(labels, handles))
            ax.legend(by_label.values(), by_label.keys(), loc='upper right', fontsize=9)
    
    plt.tight_layout()
    return fig

print("✓ Prediction plotting ready")

## Section 6: Metrics Computation

In [ ]:
def compute_event_metrics(predictions, ground_truth, tolerance_start=0.25, 
                         tolerance_end=0.25, min_overlap=0.1):
    """
    Compute event-based metrics (TP, FP, FN).
    
    Args:
        predictions: List of (start, end, prob) tuples
        ground_truth: List of (start, end) tuples
        tolerance_start: Start tolerance in seconds
        tolerance_end: End tolerance in seconds
        min_overlap: Minimum overlap ratio to count as TP
    
    Returns:
        dict: Metrics including TP, FP, FN, Sensitivity, Precision, F1
    """
    if not ground_truth:
        # No ground truth - just return detection count
        return {
            'TP': None,
            'FP': None,
            'FN': None,
            'Sensitivity': None,
            'Precision': None,
            'F1': None,
            'Total_Detections': len(predictions),
            'Ground_Truth_Count': 0
        }
    
    # Convert to start/end arrays
    pred_starts = np.array([p[0] for p in predictions])
    pred_ends = np.array([p[1] for p in predictions])
    gt_starts = np.array([g[0] for g in ground_truth])
    gt_ends = np.array([g[1] for g in ground_truth])
    
    # Track which GT events have been matched
    gt_matched = np.zeros(len(ground_truth), dtype=bool)
    tp_count = 0
    fp_count = 0
    
    # For each prediction, check if it matches a GT event
    for pred_start, pred_end in zip(pred_starts, pred_ends):
        matched = False
        
        for i, (gt_start, gt_end) in enumerate(zip(gt_starts, gt_ends)):
            if gt_matched[i]:
                continue
            
            # Check overlap
            overlap_start = max(pred_start, gt_start - tolerance_start)
            overlap_end = min(pred_end, gt_end + tolerance_end)
            
            if overlap_end > overlap_start:
                overlap_duration = overlap_end - overlap_start
                gt_duration = gt_end - gt_start
                
                if overlap_duration / gt_duration >= min_overlap:
                    # Match!
                    tp_count += 1
                    gt_matched[i] = True
                    matched = True
                    break
        
        if not matched:
            fp_count += 1
    
    # Unmatched GT events are false negatives
    fn_count = np.sum(~gt_matched)
    
    # Compute metrics
    sensitivity = tp_count / (tp_count + fn_count) if (tp_count + fn_count) > 0 else 0
    precision = tp_count / (tp_count + fp_count) if (tp_count + fp_count) > 0 else 0
    f1 = 2 * precision * sensitivity / (precision + sensitivity) if (precision + sensitivity) > 0 else 0
    
    return {
        'TP': int(tp_count),
        'FP': int(fp_count),
        'FN': int(fn_count),
        'Sensitivity': float(sensitivity),
        'Precision': float(precision),
        'F1': float(f1),
        'Total_Detections': len(predictions),
        'Ground_Truth_Count': len(ground_truth)
    }

print("✓ Metrics computation ready")

## Section 7: Data Loading Utilities

In [ ]:
def load_dataset_recording(subject_id, trial, movement, noise, sound):
    """
    Load a recording from the dataset.
    
    Returns:
        audio: (N,) audio samples (outer mic)
        imu_data: (M, 6) IMU samples
        ground_truth: List of (start, end) or None
    """
    # Load audio (outer mic only)
    audio_air, _ = load_audio(data_folder, subject_id, trial, movement, noise, sound)
    
    # Load IMU
    imu_obj = load_imu(data_folder, subject_id, trial, movement, noise, sound)
    imu_data = imu_obj.make_segment_df().values
    
    # Load ground truth if cough recording
    ground_truth = None
    if sound == Sound.COUGH:
        try:
            start_times, end_times = load_annotation(data_folder, subject_id, trial, movement, noise, sound)
            ground_truth = list(zip(start_times, end_times))
        except:
            pass
    
    return audio_air, imu_data, ground_truth

def load_uploaded_wav(file_obj):
    """
    Load WAV from Gradio file upload.
    
    Returns:
        audio: (N,) normalized audio samples
    """
    fs, audio = wavfile.read(file_obj.name)
    if fs != 16000:
        raise ValueError(f"Audio must be 16 kHz, got {fs} Hz")
    
    # Normalize by 2^29 (matching dataset preprocessing)
    audio = audio / (1 << 29)
    return audio

def load_uploaded_imu(file_obj):
    """
    Load IMU CSV from Gradio file upload.
    
    Returns:
        imu: (N, 6) IMU samples
    """
    df = pd.read_csv(file_obj.name)
    required_cols = ['Accel x', 'Accel y', 'Accel z', 'Gyro Y', 'Gyro P', 'Gyro R']
    
    if not all(col in df.columns for col in required_cols):
        raise ValueError(f"IMU CSV must contain: {required_cols}")
    
    return df[required_cols].values

print("✓ Data loading utilities ready")

## Section 8: Main Gradio Interface

In [ ]:
def run_prediction(data_source, subject_id, trial, movement, noise, sound,
                  audio_file, imu_file, modality, threshold_override):
    """
    Main prediction function called by Gradio interface.
    """
    if MODELS is None:
        return None, {"Error": "Models not loaded"}, pd.DataFrame()
    
    try:
        # Load data based on source
        if data_source == "Dataset Selector":
            if not data_folder:
                return None, {"Error": "Dataset not found"}, pd.DataFrame()
            
            # Convert dropdown values to Enum
            trial_enum = Trial(trial)
            mov_enum = Movement(movement.lower())
            noise_enum = Noise(noise.lower().replace(' ', '_'))
            sound_enum = Sound(sound.lower().replace(' ', '_'))
            
            audio, imu, ground_truth = load_dataset_recording(
                subject_id, trial_enum, mov_enum, noise_enum, sound_enum
            )
        else:  # Upload Files
            if audio_file is None or imu_file is None:
                return None, {"Error": "Please upload both audio and IMU files"}, pd.DataFrame()
            
            audio = load_uploaded_wav(audio_file)
            imu = load_uploaded_imu(imu_file)
            ground_truth = None
        
        # Map modality to model key
        modality_map = {
            "IMU-only": "imu",
            "Audio-only": "audio",
            "Multimodal": "multimodal"
        }
        model_key = modality_map[modality]
        model_data = MODELS[model_key]
        
        # Override threshold if specified (0.0 means use optimal)
        threshold = model_data['threshold'] if threshold_override == 0.0 else threshold_override
        
        # Run prediction
        raw_predictions, probs, window_times = sliding_window_predict(
            audio, imu, model_data, modality=model_key, threshold=threshold
        )
        
        # Merge detections
        predictions = merge_detections(raw_predictions, gap_threshold=0.3)
        
        # Compute metrics
        metrics = compute_event_metrics(predictions, ground_truth)
        
        # Add threshold info to metrics
        metrics['Threshold_Used'] = float(threshold)
        metrics['Is_Optimal_Threshold'] = (threshold_override == 0.0)
        
        # Create visualization
        fig = plot_predictions(audio, imu, predictions, ground_truth)
        
        # Create events table
        if predictions:
            events_df = pd.DataFrame([
                {'Start (s)': f'{s:.2f}', 'End (s)': f'{e:.2f}', 'Confidence': f'{p:.3f}'}
                for s, e, p in predictions
            ])
        else:
            events_df = pd.DataFrame({'Message': ['No coughs detected']})
        
        return fig, metrics, events_df
    
    except Exception as e:
        import traceback
        error_msg = f"Error: {str(e)}\n{traceback.format_exc()}"
        return None, {"Error": error_msg}, pd.DataFrame()

print("✓ Main prediction function ready")

In [ ]:
# Get dataset parameters for dropdowns
if data_folder:
    subject_ids = [d for d in os.listdir(data_folder) if os.path.isdir(os.path.join(data_folder, d))]
    subject_ids = sorted(subject_ids)
else:
    subject_ids = []

# Create Gradio interface
with gr.Blocks(title="Interactive Cough Detection Model Tester", theme=gr.themes.Soft()) as demo:
    gr.Markdown(
        """
        # Interactive Cough Detection Model Tester
        
        Test XGBoost cough detection models on multimodal biosignals (audio + IMU).
        
        **Instructions:**
        1. Choose data source: Dataset recordings or upload your own files
        2. Select model: IMU-only, Audio-only, or Multimodal
        3. Adjust threshold if needed (0 = use optimal from training)
        4. Click "Run Prediction" to see results
        """
    )
    
    with gr.Row():
        with gr.Column(scale=1):
            # Data source selector
            data_source = gr.Radio(
                choices=["Dataset Selector", "Upload Files"],
                value="Dataset Selector" if data_folder else "Upload Files",
                label="Data Source"
            )
            
            # Dataset selector (visible when Dataset Selector is chosen)
            with gr.Group(visible=(data_folder is not None)) as dataset_group:
                gr.Markdown("### Dataset Recording")
                subject_dropdown = gr.Dropdown(
                    choices=subject_ids,
                    value=subject_ids[0] if subject_ids else None,
                    label="Subject ID"
                )
                trial_dropdown = gr.Dropdown(
                    choices=["1", "2", "3"],
                    value="1",
                    label="Trial"
                )
                movement_dropdown = gr.Dropdown(
                    choices=["Sit", "Walk"],
                    value="Sit",
                    label="Movement"
                )
                noise_dropdown = gr.Dropdown(
                    choices=["Nothing", "Music", "Someone else cough", "Traffic"],
                    value="Nothing",
                    label="Background Noise"
                )
                sound_dropdown = gr.Dropdown(
                    choices=["Cough", "Laugh", "Deep breathing", "Throat clearing"],
                    value="Cough",
                    label="Sound Type"
                )
            
            # File upload (visible when Upload Files is chosen)
            with gr.Group(visible=(data_folder is None)) as upload_group:
                gr.Markdown("### Upload Files")
                audio_upload = gr.File(
                    label="Audio WAV (16 kHz)",
                    file_types=[".wav"]
                )
                imu_upload = gr.File(
                    label="IMU CSV (100 Hz)",
                    file_types=[".csv"]
                )
                gr.Markdown(
                    "*CSV must contain: Accel x, Accel y, Accel z, Gyro Y, Gyro P, Gyro R*"
                )
            
            # Toggle visibility based on data source
            def toggle_data_source(choice):
                if choice == "Dataset Selector":
                    return gr.update(visible=True), gr.update(visible=False)
                else:
                    return gr.update(visible=False), gr.update(visible=True)
            
            data_source.change(
                toggle_data_source,
                inputs=[data_source],
                outputs=[dataset_group, upload_group]
            )
            
            # Model selection
            gr.Markdown("### Model Settings")
            modality_radio = gr.Radio(
                choices=["IMU-only", "Audio-only", "Multimodal"],
                value="Multimodal",
                label="Model"
            )
            
            # Display optimal threshold for selected model
            if MODELS is not None:
                optimal_thresh_display = gr.Markdown(
                    f"**Optimal Threshold:** {MODELS['multimodal']['threshold']:.3f}"
                )
            else:
                optimal_thresh_display = gr.Markdown("**Optimal Threshold:** Models not loaded")
            
            # Update optimal threshold display when model changes
            def update_optimal_threshold(modality):
                if MODELS is None:
                    return "**Optimal Threshold:** Models not loaded"
                model_key = modality.lower().replace('-only', '')
                thresh = MODELS[model_key]['threshold']
                return f"**Optimal Threshold:** {thresh:.3f}"
            
            modality_radio.change(
                update_optimal_threshold,
                inputs=[modality_radio],
                outputs=[optimal_thresh_display]
            )
            
            threshold_slider = gr.Slider(
                minimum=0.0,
                maximum=1.0,
                value=0.0,
                step=0.05,
                label="Threshold Override",
                info="Set to 0.0 to use optimal threshold above, or override with custom value"
            )
            
            # Run button
            run_btn = gr.Button(
                "Run Prediction",
                variant="primary",
                size="lg"
            )
        
        with gr.Column(scale=2):
            # Outputs
            plot_output = gr.Plot(label="Waveform with Detections")
            metrics_output = gr.JSON(label="Metrics")
            events_output = gr.Dataframe(label="Detected Events")
    
    # Connect button to prediction function
    run_btn.click(
        run_prediction,
        inputs=[
            data_source, subject_dropdown, trial_dropdown, movement_dropdown,
            noise_dropdown, sound_dropdown, audio_upload, imu_upload,
            modality_radio, threshold_slider
        ],
        outputs=[plot_output, metrics_output, events_output]
    )

print("✓ Gradio interface created")

## Section 9: Launch Application

In [ ]:
# Launch Gradio app
if MODELS is not None:
    print("\n" + "="*70)
    print("Launching Interactive Cough Detection Model Tester...")
    print("="*70)
    demo.launch(share=False, debug=True)
else:
    print("\n" + "="*70)
    print("Cannot launch: Models not loaded")
    print("Please run Model_Training_XGBoost.ipynb first to train models")
    print("="*70)

## Section 10: Usage Examples

### Example 1: Test on Dataset Recording

1. Select "Dataset Selector" as data source
2. Choose Subject: `14287`, Trial: `1`, Movement: `Sit`, Noise: `Nothing`, Sound: `Cough`
3. Select Model: `Multimodal`
4. Keep threshold at `0.0` (auto-optimal)
5. Click "Run Prediction"

**Expected output:**
- Waveform plot with red prediction spans and green ground truth spans
- Metrics showing TP/FP/FN counts, Sensitivity ~0.9+, Precision ~0.8+
- Events table listing detected cough times with confidence scores

### Example 2: Compare Models

Run the same recording through all three models:
- IMU-only: Lower sensitivity, may miss some coughs
- Audio-only: Good performance on coughs
- Multimodal: Best overall performance

### Example 3: Threshold Adjustment

1. Run prediction with threshold `0.0` (optimal)
2. Increase threshold to `0.7` - fewer detections, higher precision
3. Decrease threshold to `0.3` - more detections, lower precision

### Example 4: Test on Non-Cough Sounds

1. Select Sound: `Laugh` or `Throat clearing`
2. Model should show low/no detections (good specificity)
3. No ground truth will be shown (only available for coughs)

### Example 5: Upload Custom Files

1. Export a recording from the dataset as WAV + CSV
2. Select "Upload Files" as data source
3. Upload both files
4. Run prediction (no ground truth comparison available)

## Performance Notes

- **Processing time**: ~2-5 seconds for 10-second recording (depends on hardware)
- **Window size**: 0.4 seconds (fixed, from training)
- **Hop size**: 0.05 seconds (50ms overlap between windows)
- **Multimodal model**: Best performance but requires both audio and IMU
- **IMU-only**: Useful for privacy-preserving scenarios (no audio)
- **Audio-only**: Strong baseline, works well in quiet environments

## Limitations

1. **Dataset bias**: Models trained on 15 subjects, may not generalize to all populations
2. **Microphone dependency**: Audio features tuned to specific hardware
3. **Fixed window**: 0.4s windows may miss very long/short coughs
4. **Threshold sensitivity**: Performance varies with threshold choice
5. **No real-time processing**: Batch processing only (not streaming)

## Next Steps

- Test on different subjects to assess generalization
- Experiment with threshold values for your use case
- Compare model performance across different noise conditions
- Analyze false positives/negatives to understand model weaknesses
- Consider deploying to edge device for real-time monitoring